# Pharma SFA Agent — Capstone Project (Kaggle Notebook)

**Track:** Enterprise Agents  
**Author:** Ashish Ate  
**Date:** 2025-12-01

## Abstract
This notebook implements a multi-agent Sales Force Automation (SFA) prototype for a pharmaceutical field force. It includes:
- Synthetic dataset creation (doctors, MRs, hospitals, visits, RCPA),
- A Classifier Agent (doctor revenue segmentation: High/Mid/Low),
- A Molecule Recommender Agent (therapy → molecules mapping),
- A Scheduler Agent ( 7-day MR visit scheduler),
- An Orchestrator Agent that runs the pipeline end-to-end,
- A Streamlit app saved from the notebook for a local demo (Physician Visit plan +Travel map +MR level summary + MR Note + visit form + RCPA form + dashboard),
- Outputs (CSV & Excel) for judges and demonstration.


## How to run
1. Run cells sequentially.  
2. After running the pipeline, `output/next_7day_plan.csv` will be created.  
3. The Streamlit app file `streamlit_app.py` will be created in the notebook workspace (run locally with `streamlit run streamlit_app.py`).

In [1]:
# Basic Imports
import os
import uuid
from datetime import datetime, date, timedelta, time
import random
from pathlib import Path
import pandas as pd
import numpy as np

# ensure data dir exists
DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

# ensure output dir exists (all generated outputs will go here)
OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(exist_ok=True)

## Configuration of molecule under theray area and segmentation  
Therapy area → molecule mapping and simple segmentation thresholds are defined below.

In [2]:
# Config: therapy -> molecules and segmentation thresholds
therapy_molecule_map = {
    "Cardiology": ["Apixaban","Rivaroxaban","Dabigatran","Clopidogrel","Aspirin EC","Atorvastatin","Rosuvastatin","Ezetimibe","Metoprolol","Carvedilol","Amlodipine","Telmisartan","Olmesartan","Losartan","Ivabradine","Sacubitril/Valsartan","Furosemide","Spironolactone"],
    "Respiratory": ["Budesonide+Formoterol","Fluticasone+Salmeterol","Montelukast","Levocetirizine","Doxofylline","Theophylline","Tiotropium","Glycopyrronium","Salbutamol","Beclomethasone","Azithromycin"],
    "Diabetology": ["Metformin","Vildagliptin","Sitagliptin","Teneligliptin","Empagliflozin","Dapagliflozin","Glimepiride","Insulin Glargine","Insulin Aspart","Pioglitazone"],
    "Neurology": ["Pregabalin","Duloxetine","Amitriptyline","Sertraline","Escitalopram","Olanzapine","Risperidone","Quetiapine","Valproate","Carbamazepine"],
    "Infectious Disease": ["Amox+Clav","Cefixime","Ofloxacin+Ornidazole","Ciprofloxacin","Ceftriaxone","Azithromycin","Doxycycline","Linezolid"],
    "Orthopedics": ["Etodolac","Aceclofenac","Diclofenac","Etoricoxib","Thiocolchicoside","Tizanidine","Calcitriol+Calcium+Zinc","Vitamin D3"],
    "Gastroenterology": ["Pantoprazole","Rabeprazole","Esomeprazole","Domperidone","Itopride","Ondansetron","UDCA","Pancreatin"],
    "Gynaecology": ["Dydrogesterone","Progesterone SG","Letrozole","Folic Acid+Iron","L-Arginine","Myoinositol","Estradiol Valerate"],
    "Dermatology": ["Isotretinoin","Adapalene+Clindamycin","Ketoconazole","Terbinafine","Itraconazole","Tacrolimus"],
    "Pediatrics": ["Paracetamol Suspension","Cefpodoxime Dry Syrup","Zinc+ORS","Montelukast+Levocetirizine Syrup"],
    "Nephrology": ["Sevelamer","Ferric Carboxymaltose","Erythropoietin"],
    "Oncology": ["Zoledronic Acid","Filgrastim","Ondansetron (High Dose)"]
}

SEGMENT_THRESHOLDS = {"high":100, "mid":40}  # High >=100, Mid 40-99, Low <40

## Dataset creation
we will create master tables:
- doctors_master.csv
- mr_reps.csv
- hospitals_master.csv
- physician_visit_table.csv (historical visits)
- rcpa_chemist_table.csv (chemist RCPA snapshots)

The synthetic data is realistic for a pharma SFA context and usable by the agents.


In [3]:
# Dataset Creation 
import pandas as pd
from pathlib import Path

DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

# Hospitals 
hospitals = [
    ("H001","Fortis Hospital Mohali", "Mohali", 30.67995, 76.72211),
    ("H002","Ivy Hospital Mohali", "Mohali", 30.706955, 76.708018),
    ("H003","Max Super Speciality Hospital Mohali", "Mohali", 30.73903, 76.71506),
    ("H004","Civil Hospital Mohali", "Mohali", 30.7397742, 76.7138567),
    ("H005","Ace Heart Institute Mohali", "Mohali", 30.7045, 76.7330),
]
df_hospitals = pd.DataFrame(hospitals, columns=["hospital_id","hospital_name","city","geo_lat","geo_long"])
df_hospitals.to_csv(DATA_DIR/"hospitals_master.csv", index=False)

# Doctors
doctors = [
    ("D001","Dr. R. Sharma","Cardiologist","Cardiology","H001",120),
    ("D002","Dr. S. Verma","Physician","General Medicine","H002",40),
    ("D003","Dr. A. Kapoor","Endocrinologist","Diabetology","H003",85),
    ("D004","Dr. P. Iyer","Pulmonologist","Respiratory","H004",65),
    ("D005","Dr. N. Mehta","Neurologist","Neurology","H005",30),
    ("D006","Dr. M. Rao","Cardiologist","Cardiology","H001",95),
    ("D007","Dr. V. Ghosh","Orthopedician","Orthopedics","H002",22),
    ("D008","Dr. L. Bose","Pediatrician","Pediatrics","H003",55),
    ("D009","Dr. K. Iqbal","Gastroenterologist","Gastroenterology","H002",28),
    ("D010","Dr. T. Singh","Nephrologist","Nephrology","H004",18),
    ("D011","Dr. R. Desai","Cardiologist","Cardiology","H003",130),
    ("D012","Dr. S. Mukherjee","Physician","General Medicine","H004",47),
    ("D013","Dr. A. Bhattacharya","Gynaecologist","Gynaecology","H005",36),
    ("D014","Dr. F. Khan","Dermatologist","Dermatology","H001",20),
    ("D015","Dr. Y. Patel","Pulmonologist","Respiratory","H004",70),
]
df_doctors = pd.DataFrame(doctors, columns=["doctor_id","doctor_name","doctor_speciality","therapy_area","hospital_id","avg_weekly_rx"])
df_doctors.to_csv(DATA_DIR/"doctors_master.csv", index=False)

# MRs
mrs = [
    ("M001","Mr. Ankit Jain","North Zone","MG01"),
    ("M002","Ms. Priya Nair","East Zone","MG01"),
    ("M003","Mr. Rohit Das","South Zone","MG02"),
    ("M004","Ms. Nisha Roy","West Zone","MG02"),
]
df_mrs = pd.DataFrame(mrs, columns=["mr_id","mr_name","territory","manager_id"])
df_mrs.to_csv(DATA_DIR/"mr_reps.csv", index=False)

# Physician visit history
visits = [
    ("V0001","M001","2025-11-20","10:15","D001","Apixaban 5 mg","Stroke prevention","Yes|2","Yes",120,"Rivaroxaban","Price",12,True,0.75,0.82,"Follow-up","Completed","Approved","H001","Good"),
    ("V0002","M001","2025-11-21","15:00","D006","Rosuvastatin 20 mg","Lipid control","Yes|1","Yes",95,"Atorvastatin","Prefers competitor",10,False,0.68,0.79,"Clinical brief","Completed","Approved","H001","Ok"),
    ("V0003","M002","2025-11-18","11:00","D003","Empagliflozin","SGLT2i","Yes|0","No",85,"Dapagliflozin","Renal concern",9,True,0.62,0.70,"Renal leaflet","Completed","Approved","H003","Needs data"),
]
cols_visits = ["visit_id","mr_id","visit_date","visit_time","doctor_id","product_detailed","core_message_shared","samples_provided","rx_commitment","expected_rx_volume","competitor_mentioned","objections_noted","call_duration_min","digital_engagement","sentiment_score","ai_call_score","next_best_action","visit_status","approval_status","hospital_id","manager_comments"]
df_visits = pd.DataFrame(visits, columns=cols_visits)
df_visits = df_visits.merge(df_hospitals[["hospital_id","geo_lat","geo_long"]], on="hospital_id", how="left")
df_visits.to_csv(DATA_DIR/"physician_visit_table.csv", index=False)

# RCPA table
rcpa = [
    ("R0001","2025-11-20","M001","C001","HealthPlus Meds","Mohali","Apixaban","5 mg",450,15,12,"Rivaroxaban",420,8,"In stock",18,"Competitor discount",25,"https://example.com/R1"),
    ("R0002","2025-11-21","M002","C002","CareWell Pharmacy","Mohali","Rosuvastatin","20 mg",90,10,9,"Atorvastatin",85,7,"Low stock",22,"Local brand preferred",12,"https://example.com/R2"),
]
cols_rcpa = ["rcpa_id","visit_date","mr_id","chemist_id","chemist_name","town_city","your_brand_name","your_strength","your_mrp","your_rx_captured","your_conversion_done","comp_brand_name","comp_mrp","comp_rx_share","stock_availability","avg_margin_percent","lost_business_reason","weekly_demand_units","photo_proof_url"]
df_rcpa = pd.DataFrame(rcpa, columns=cols_rcpa)
df_rcpa.to_csv(DATA_DIR/"rcpa_chemist_table.csv", index=False)

print("Data written to /data")


Data written to /data


## Classifier Agent
Simple threshold-based classifier that assigns segment labels:
- High: avg_weekly_rx >= 100
- Mid: 40 <= avg_weekly_rx < 100
- Low: avg_weekly_rx < 40

In [4]:
# Classifier Agent
def classify_doctor(expected_rx_volume):
    if expected_rx_volume >= SEGMENT_THRESHOLDS["high"]:
        return "High"
    if expected_rx_volume >= SEGMENT_THRESHOLDS["mid"]:
        return "Mid"
    return "Low"

def classify_doctors(df):
    df2 = df.copy()
    df2["segment"] = df2["avg_weekly_rx"].apply(classify_doctor)
    return df2

df_doctors_seg = classify_doctors(df_doctors)
df_doctors_seg.head(10)

,doctor_id,doctor_name,doctor_speciality,therapy_area,hospital_id,avg_weekly_rx,segment
0,D001,Dr. R. Sharma,Cardiologist,Cardiology,H001,120,High
1,D002,Dr. S. Verma,Physician,General Medicine,H002,40,Mid
2,D003,Dr. A. Kapoor,Endocrinologist,Diabetology,H003,85,Mid
3,D004,Dr. P. Iyer,Pulmonologist,Respiratory,H004,65,Mid
4,D005,Dr. N. Mehta,Neurologist,Neurology,H005,30,Low
5,D006,Dr. M. Rao,Cardiologist,Cardiology,H001,95,Mid
6,D007,Dr. V. Ghosh,Orthopedician,Orthopedics,H002,22,Low
7,D008,Dr. L. Bose,Pediatrician,Pediatrics,H003,55,Mid
8,D009,Dr. K. Iqbal,Gastroenterologist,Gastroenterology,H002,28,Low
9,D010,Dr. T. Singh,Nephrologist,Nephrology,H004,18,Low


## Molecule Recommender Agent
Uses `therapy_molecule_map` to return a short list of molecules to detail for each doctor.
In production we would rank candidates by: RCPA share, objections, competitor presence, clinical fit.

In [5]:
# Recommender Agent 
def recommend_molecules(therapy_area, top_n=3, seed=None):
    candidates = therapy_molecule_map.get(therapy_area, [])
    if not candidates:
        return []
    rng = random.Random(seed)
    chosen = rng.sample(candidates, min(len(candidates), top_n))
    return chosen

# Example
df_doctors_seg["recommended_molecules"] = df_doctors_seg["therapy_area"].apply(lambda ta: recommend_molecules(ta, top_n=3))
df_doctors_seg[["doctor_id","doctor_name","therapy_area","avg_weekly_rx","segment","recommended_molecules"]].head(10)

,doctor_id,doctor_name,therapy_area,avg_weekly_rx,segment,recommended_molecules
0,D001,Dr. R. Sharma,Cardiology,120,High,"[Apixaban, Spironolactone, Furosemide]"
1,D002,Dr. S. Verma,General Medicine,40,Mid,[]
2,D003,Dr. A. Kapoor,Diabetology,85,Mid,"[Glimepiride, Empagliflozin, Insulin Glargine]"
3,D004,Dr. P. Iyer,Respiratory,65,Mid,"[Azithromycin, Beclomethasone, Fluticasone+Sal..."
4,D005,Dr. N. Mehta,Neurology,30,Low,"[Risperidone, Olanzapine, Sertraline]"
5,D006,Dr. M. Rao,Cardiology,95,Mid,"[Aspirin EC, Rivaroxaban, Atorvastatin]"
6,D007,Dr. V. Ghosh,Orthopedics,22,Low,"[Thiocolchicoside, Diclofenac, Tizanidine]"
7,D008,Dr. L. Bose,Pediatrics,55,Mid,"[Montelukast+Levocetirizine Syrup, Cefpodoxime..."
8,D009,Dr. K. Iqbal,Gastroenterology,28,Low,"[Pancreatin, Rabeprazole, UDCA]"
9,D010,Dr. T. Singh,Nephrology,18,Low,"[Erythropoietin, Sevelamer, Ferric Carboxymalt..."


## Scheduler Agent
Greedy scheduler that:
- Prioritizes High → Mid → Low doctors,
- Assigns timeslots (30-min slots between 09:00–17:00),
- Round-robin assigns MRs to create balanced load.
This is intentionally simple and easy to explain to judges; you can extend with travel-time optimization (TSP), MR availability, and constraints.


In [6]:
# Scheduler agent implementation
SLOT_MINUTES = 30
START_HOUR = 9
END_HOUR = 17

def generate_slots_for_days(start_date: date, days=7):
    slots = []
    for day_offset in range(days):
        day = start_date + timedelta(days=day_offset)
        current = datetime.combine(day, time(hour=START_HOUR))
        end_dt = datetime.combine(day, time(hour=END_HOUR))
        while current < end_dt:
            slots.append(current)
            current += timedelta(minutes=SLOT_MINUTES)
    return slots

def schedule_visits(df_docs_segmented, mr_list, start_date=date.today(), days=7):
    # Convert segment to numeric priority
    seg_priority = {"High":2,"Mid":1,"Low":0}
    df_sorted = df_docs_segmented.copy()
    df_sorted["seg_priority"] = df_sorted["segment"].map(seg_priority)
    df_sorted = df_sorted.sort_values(by=["seg_priority","avg_weekly_rx"], ascending=[False, False])
    
    slots = generate_slots_for_days(start_date, days)
    assignments = []
    mr_count = len(mr_list)
    slot_idx = 0
    for _, doc in df_sorted.iterrows():
        if slot_idx >= len(slots):
            break
        slot_dt = slots[slot_idx]
        assigned_mr = mr_list[slot_idx % mr_count]["mr_id"]
        assignments.append({
            "doctor_id": doc["doctor_id"],
            "doctor_name": doc["doctor_name"],
            "therapy_area": doc["therapy_area"],
            "segment": doc["segment"],
            "assigned_mr": assigned_mr,
            "scheduled_date": slot_dt.strftime("%Y-%m-%d"),
            "scheduled_time": slot_dt.strftime("%H:%M")
        })
        slot_idx += 1
    return pd.DataFrame(assignments)

# Demo schedule
mr_list = df_mrs.to_dict(orient="records")
plan_df = schedule_visits(df_doctors_seg, mr_list, start_date=date.today(), days=7)
plan_df.head(10)

,doctor_id,doctor_name,therapy_area,segment,assigned_mr,scheduled_date,scheduled_time
0,D011,Dr. R. Desai,Cardiology,High,M001,2025-12-01,09:00
1,D001,Dr. R. Sharma,Cardiology,High,M002,2025-12-01,09:30
2,D006,Dr. M. Rao,Cardiology,Mid,M003,2025-12-01,10:00
3,D003,Dr. A. Kapoor,Diabetology,Mid,M004,2025-12-01,10:30
4,D015,Dr. Y. Patel,Respiratory,Mid,M001,2025-12-01,11:00
5,D004,Dr. P. Iyer,Respiratory,Mid,M002,2025-12-01,11:30
6,D008,Dr. L. Bose,Pediatrics,Mid,M003,2025-12-01,12:00
7,D012,Dr. S. Mukherjee,General Medicine,Mid,M004,2025-12-01,12:30
8,D002,Dr. S. Verma,General Medicine,Mid,M001,2025-12-01,13:00
9,D013,Dr. A. Bhattacharya,Gynaecology,Low,M002,2025-12-01,13:30


## Orchestrator Agent
Runs the pipeline:
1. Reads masters
2. Classifies doctors
3. Recommends molecules
4. Schedules visits for 7 days
5. Writes `output/next_7day_plan.csv`

In [7]:
# Orchestrator
def run_pipeline(start_date=date.today()):
    # masters already in df_* variables, but reading from disk to simulate production
    df_docs = pd.read_csv(DATA_DIR/"doctors_master.csv")
    df_mrs_local = pd.read_csv(DATA_DIR/"mr_reps.csv")
    
    df_docs_seg = classify_doctors(df_docs)
    df_docs_seg["recommended_molecules"] = df_docs_seg["therapy_area"].apply(lambda ta: recommend_molecules(ta, top_n=3))
    
    mr_list_local = df_mrs_local.to_dict(orient="records")
    plan = schedule_visits(df_docs_seg, mr_list_local, start_date=start_date, days=7)
    # join recommended molecules
    plan = plan.merge(df_docs_seg[["doctor_id","recommended_molecules"]], on="doctor_id", how="left")
    plan.to_csv(OUTPUT_DIR / "next_7day_plan.csv", index=False)
    return plan

plan = run_pipeline()
print("Plan written to output/next_7day_plan.csv")
plan.head(12)


Plan written to output/next_7day_plan.csv


,doctor_id,doctor_name,therapy_area,segment,assigned_mr,scheduled_date,scheduled_time,recommended_molecules
0,D011,Dr. R. Desai,Cardiology,High,M001,2025-12-01,09:00,"[Sacubitril/Valsartan, Rivaroxaban, Aspirin EC]"
1,D001,Dr. R. Sharma,Cardiology,High,M002,2025-12-01,09:30,"[Furosemide, Rosuvastatin, Carvedilol]"
2,D006,Dr. M. Rao,Cardiology,Mid,M003,2025-12-01,10:00,"[Sacubitril/Valsartan, Rivaroxaban, Furosemide]"
3,D003,Dr. A. Kapoor,Diabetology,Mid,M004,2025-12-01,10:30,"[Vildagliptin, Sitagliptin, Dapagliflozin]"
4,D015,Dr. Y. Patel,Respiratory,Mid,M001,2025-12-01,11:00,"[Azithromycin, Beclomethasone, Fluticasone+Sal..."
5,D004,Dr. P. Iyer,Respiratory,Mid,M002,2025-12-01,11:30,"[Theophylline, Azithromycin, Doxofylline]"
6,D008,Dr. L. Bose,Pediatrics,Mid,M003,2025-12-01,12:00,"[Montelukast+Levocetirizine Syrup, Cefpodoxime..."
7,D012,Dr. S. Mukherjee,General Medicine,Mid,M004,2025-12-01,12:30,[]
8,D002,Dr. S. Verma,General Medicine,Mid,M001,2025-12-01,13:00,[]
9,D013,Dr. A. Bhattacharya,Gynaecology,Low,M002,2025-12-01,13:30,"[Dydrogesterone, Progesterone SG, Folic Acid+I..."


## Plan Analysis
We show:
- counts by segment
- MR workload
- therapy area distribution


In [8]:
# Final MR plan export cell (territory speeds, expected end time, extra columns, folium map)
# Run this cell to generate final CSV/JSON/metrics and folium map.
import matplotlib.pyplot as plt
import json
from math import radians, sin, cos, sqrt, atan2
import numpy as np
from datetime import timedelta
from pathlib import Path
import uuid

# Ensure OUTPUT_DIR exists (defined earlier at top of notebook)
OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(exist_ok=True)

DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

# Territory-specific average speeds (km/h)
AVG_SPEED_MAP = {
    "North Zone": 20.0,
    "East Zone": 30.0,
    "South Zone": 35.0,
    "West Zone": 30.0,
    "default": 30.0
}

# Load latest plan + masters
# NOTE: plan is produced by the orchestrator and saved under output/
plan_path = OUTPUT_DIR / "next_7day_plan.csv"
if not plan_path.exists():
    raise FileNotFoundError(f"Expected plan at {plan_path} — run the orchestrator cell first.")

plan = pd.read_csv(plan_path)
doctors_master = pd.read_csv(DATA_DIR / "doctors_master.csv")
hospitals_master = pd.read_csv(DATA_DIR / "hospitals_master.csv")
mrs = pd.read_csv(DATA_DIR / "mr_reps.csv")
df_visits_hist = pd.read_csv(DATA_DIR / "physician_visit_table.csv")

# Ensure recommended_molecules exists
if "recommended_molecules" not in plan.columns:
    try:
        docs_seg = pd.read_csv(DATA_DIR / "doctors_segmented.csv")
        plan = plan.merge(docs_seg[["doctor_id", "recommended_molecules"]], on="doctor_id", how="left")
    except Exception:
        plan["recommended_molecules"] = None

# Normalize recommended_molecules to list
def normalize_rec(x):
    if pd.isna(x) or x is None:
        return []
    if isinstance(x, (list, tuple)):
        return list(x)
    # try eval, else split by semicolon or comma
    try:
        val = eval(x)
        if isinstance(val, (list, tuple)):
            return list(val)
    except:
        pass
    if isinstance(x, str) and (";" in x):
        return [s.strip() for s in x.split(";") if s.strip()]
    if isinstance(x, str) and ("," in x):
        return [s.strip() for s in x.strip("[]").split(",") if s.strip()]
    return [str(x)]

plan["recommended_molecules_list"] = plan["recommended_molecules"].apply(normalize_rec)

# Build plan_geo with geo coords and datetime
plan_geo = plan.merge(doctors_master[["doctor_id","doctor_name","hospital_id"]], on="doctor_id", how="left")
plan_geo = plan_geo.merge(hospitals_master[["hospital_id","geo_lat","geo_long"]], on="hospital_id", how="left")
plan_geo["sched_dt"] = pd.to_datetime(plan_geo["scheduled_date"] + " " + plan_geo["scheduled_time"])

# Haversine helper
def haversine(lat1, lon1, lat2, lon2):
    if any(pd.isna([lat1,lon1,lat2,lon2])):
        return np.nan
    R = 6371.0
    phi1, phi2 = radians(lat1), radians(lat2)
    dphi = radians(lat2 - lat1)
    dlambda = radians(lon2 - lon1)
    a = sin(dphi/2)**2 + cos(phi1)*cos(phi2)*sin(dlambda/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

# Create pairwise travel estimates and a lookup for prev -> current travel
pair_travel_rows = []
prev_lookup = {}  # key: (mr_id, current_doctor_id) -> (dist_km, est_min)
for mr_id, grp in plan_geo.sort_values(["assigned_mr","sched_dt"]).groupby("assigned_mr"):
    grp = grp.reset_index(drop=True)
    territory = None
    if mr_id in mrs["mr_id"].values:
        territory = mrs.loc[mrs["mr_id"]==mr_id, "territory"].values[0]
    speed = AVG_SPEED_MAP.get(territory, AVG_SPEED_MAP["default"])
    for i in range(1, len(grp)):
        r1 = grp.loc[i-1]
        r2 = grp.loc[i]
        dist_km = haversine(r1["geo_lat"], r1["geo_long"], r2["geo_lat"], r2["geo_long"])
        est_min = None if pd.isna(dist_km) else (dist_km / speed) * 60.0
        pair = {
            "assigned_mr": mr_id,
            "mr_territory": territory,
            "avg_speed_kmph": speed,
            "from_doctor": r1["doctor_id"],
            "to_doctor": r2["doctor_id"],
            "from_dt": r1["sched_dt"].isoformat(),
            "to_dt": r2["sched_dt"].isoformat(),
            "distance_km": float(dist_km) if not pd.isna(dist_km) else None,
            "est_travel_min": float(est_min) if est_min is not None else None
        }
        pair_travel_rows.append(pair)
        # store for quick lookup: travel from r1 to r2
        prev_lookup[(mr_id, r2["doctor_id"])] = (pair["distance_km"], pair["est_travel_min"])

df_pair_travel = pd.DataFrame(pair_travel_rows)
df_pair_travel.to_csv(OUTPUT_DIR / "pair_travel_estimates.csv", index=False)

# Doctor-level avg call duration from history (if exists)
doc_avg_dur = {}
if "call_duration_min" in df_visits_hist.columns:
    df_visits_hist.rename(columns={c:c.strip() for c in df_visits_hist.columns}, inplace=True)
    doc_avg = df_visits_hist.groupby("doctor_id")["call_duration_min"].agg(lambda x: float(np.nanmean(x.dropna()))).to_dict()
    doc_avg_dur = {k: float(v) for k,v in doc_avg.items()}

segment_default_duration = {"High":15, "Mid":10, "Low":7}

# Build final MR plan rows with extra columns
final_rows = []
for idx, row in plan.sort_values(["assigned_mr","scheduled_date","scheduled_time"]).iterrows():
    slot_uuid = str(uuid.uuid4())[:8]
    doc_id = row["doctor_id"]
    seg = row.get("segment", None)
    call_dur = doc_avg_dur.get(doc_id, None)
    if call_dur is None:
        call_dur = segment_default_duration.get(seg, 10)
    rec_mols = row.get("recommended_molecules_list", [])
    # travel estimate from previous if available
    prev_key = (row["assigned_mr"], doc_id)
    est_km, est_min = prev_lookup.get(prev_key, (None, None))
    # territory & speed
    territory = None
    if row["assigned_mr"] in mrs["mr_id"].values:
        territory = mrs.loc[mrs["mr_id"]==row["assigned_mr"], "territory"].values[0]
    speed_for_row = AVG_SPEED_MAP.get(territory, AVG_SPEED_MAP["default"])
    # scheduled datetimes
    start_dt = pd.to_datetime(row["scheduled_date"] + " " + row["scheduled_time"])
    end_dt = start_dt + timedelta(minutes=float(call_dur))
    final_rows.append({
        "slot_id": slot_uuid,
        "assigned_mr": row["assigned_mr"],
        "mr_name": mrs.loc[mrs["mr_id"]==row["assigned_mr"], "mr_name"].values[0] if row["assigned_mr"] in mrs["mr_id"].values else row["assigned_mr"],
        "territory": territory,
        "avg_speed_kmph": speed_for_row,
        "doctor_id": doc_id,
        "doctor_name": doctors_master.loc[doctors_master["doctor_id"]==doc_id, "doctor_name"].values[0] if doc_id in doctors_master["doctor_id"].values else None,
        "therapy_area": row.get("therapy_area", None),
        "molecules_list": "; ".join([str(x) for x in rec_mols]) if rec_mols else None,
        "molecules_json": json.dumps(rec_mols, ensure_ascii=False),
        "segment": seg,
        "scheduled_date": row["scheduled_date"],
        "scheduled_time": row["scheduled_time"],
        "visit_slot_start": start_dt.isoformat(),
        "visit_slot_end": end_dt.isoformat(),
        "expected_end_time": end_dt.strftime("%Y-%m-%d %H:%M"),
        "call_duration_min": float(call_dur),
        "estimated_travel_km_from_prev": float(est_km) if est_km is not None else None,
        "estimated_travel_min_from_prev": float(est_min) if est_min is not None else None,
        "mr_notes": ""
    })

final_mr_plan = pd.DataFrame(final_rows)

# Save final CSV and JSON into output/
final_mr_plan.to_csv(OUTPUT_DIR / "next_7day_mr_visit.csv", index=False)
final_mr_plan.to_json(OUTPUT_DIR / "next_7day_mr_visit_full.json", orient="records", force_ascii=False)

# Metrics summary
metrics = {
    "total_scheduled_visits": int(len(plan)),
    "total_doctors_scheduled": int(plan["doctor_id"].nunique()),
    "total_doctors": int(doctors_master.shape[0]),
    "segment_counts": plan["segment"].value_counts().to_dict(),
    "mr_counts": plan["assigned_mr"].value_counts().to_dict()
}
with open(OUTPUT_DIR / "plan_metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)
pd.DataFrame([{"metric":k, "value":json.dumps(v) if isinstance(v, dict) else v} for k,v in metrics.items()]).to_csv(OUTPUT_DIR / "plan_metrics.csv", index=False)

# Attempt folium map (optional)
map_file = OUTPUT_DIR / "mr_travel_map.html"
try:
    import folium
    valid_coords = hospitals_master.dropna(subset=["geo_lat","geo_long"])
    if not valid_coords.empty and "sched_dt" in plan_geo.columns:
        center_lat = float(valid_coords["geo_lat"].mean())
        center_lon = float(valid_coords["geo_long"].mean())
        fmap = folium.Map(location=[center_lat, center_lon], zoom_start=10)
        colors = ['red','blue','green','purple','orange','darkred','lightblue','cadetblue']
        for i, (mr_id, grp) in enumerate(plan_geo.sort_values(["assigned_mr","sched_dt"]).groupby("assigned_mr")):
            coords = []
            for _, r in grp.iterrows():
                if not pd.isna(r["geo_lat"]) and not pd.isna(r["geo_long"]):
                    coords.append((r["geo_lat"], r["geo_long"]))
                    folium.CircleMarker(
                        location=(r["geo_lat"], r["geo_long"]),
                        radius=4,
                        color=colors[i % len(colors)],
                        fill=True,
                        fill_opacity=0.7,
                        popup=f"{mr_id} | {r['doctor_id']} | {r['scheduled_date']} {r['scheduled_time']}"
                    ).add_to(fmap)
            if len(coords) >= 2:
                folium.PolyLine(coords, color=colors[i % len(colors)], weight=3, opacity=0.7).add_to(fmap)
        fmap.save(str(map_file))
        print(f"MAP_SAVED_PATH:: {map_file.resolve()}")
        print(f"Folium map saved to: {map_file} (open in browser to view)")
    else:
        print("No hospital geo coords available for folium map.")
except Exception as e:
    print("Folium not available or failed — install folium (`pip install folium`) to enable map and re-run.")
    print("Folium error:", e)

# Final prints and display
print("Saved final CSV:", OUTPUT_DIR / "next_7day_mr_visit.csv")
print("Saved final JSON:", OUTPUT_DIR / "next_7day_mr_visit_full.json")
print("Saved pairwise travel CSV:", OUTPUT_DIR / "pair_travel_estimates.csv")
print("Saved metrics:", OUTPUT_DIR / "plan_metrics.json", "and", OUTPUT_DIR / "plan_metrics.csv")
display(final_mr_plan.head(40))


MAP_SAVED_PATH:: C:\Users\ashis\OneDrive\Desktop\sales force automation\output\mr_travel_map.html
Folium map saved to: output\mr_travel_map.html (open in browser to view)
Saved final CSV: output\next_7day_mr_visit.csv
Saved final JSON: output\next_7day_mr_visit_full.json
Saved pairwise travel CSV: output\pair_travel_estimates.csv
Saved metrics: output\plan_metrics.json and output\plan_metrics.csv


,slot_id,assigned_mr,mr_name,territory,avg_speed_kmph,doctor_id,doctor_name,therapy_area,molecules_list,molecules_json,segment,scheduled_date,scheduled_time,visit_slot_start,visit_slot_end,expected_end_time,call_duration_min,estimated_travel_km_from_prev,estimated_travel_min_from_prev,mr_notes
0,f1e9bed3,M001,Mr. Ankit Jain,North Zone,20.0,D011,Dr. R. Desai,Cardiology,Sacubitril/Valsartan; Rivaroxaban; Aspirin EC,"[""Sacubitril/Valsartan"", ""Rivaroxaban"", ""Aspir...",High,2025-12-01,09:00,2025-12-01T09:00:00,2025-12-01T09:15:00,2025-12-01 09:15,15.0,NaN,NaN,
1,25b0379c,M001,Mr. Ankit Jain,North Zone,20.0,D015,Dr. Y. Patel,Respiratory,Azithromycin; Beclomethasone; Fluticasone+Salm...,"[""Azithromycin"", ""Beclomethasone"", ""Fluticason...",Mid,2025-12-01,11:00,2025-12-01T11:00:00,2025-12-01T11:10:00,2025-12-01 11:10,10.0,0.141680,0.425040,
2,eda9868b,M001,Mr. Ankit Jain,North Zone,20.0,D002,Dr. S. Verma,General Medicine,None,[],Mid,2025-12-01,13:00,2025-12-01T13:00:00,2025-12-01T13:10:00,2025-12-01 13:10,10.0,3.691759,11.075277,
3,7270449c,M001,Mr. Ankit Jain,North Zone,20.0,D007,Dr. V. Ghosh,Orthopedics,Etodolac; Vitamin D3; Calcitriol+Calcium+Zinc,"[""Etodolac"", ""Vitamin D3"", ""Calcitriol+Calcium...",Low,2025-12-01,15:00,2025-12-01T15:00:00,2025-12-01T15:07:00,2025-12-01 15:07,7.0,0.000000,0.000000,
4,6ad06b35,M002,Ms. Priya Nair,East Zone,30.0,D001,Dr. R. Sharma,Cardiology,Furosemide; Rosuvastatin; Carvedilol,"[""Furosemide"", ""Rosuvastatin"", ""Carvedilol""]",High,2025-12-01,09:30,2025-12-01T09:30:00,2025-12-01T09:42:00,2025-12-01 09:42,12.0,NaN,NaN,
5,5480ae61,M002,Ms. Priya Nair,East Zone,30.0,D004,Dr. P. Iyer,Respiratory,Theophylline; Azithromycin; Doxofylline,"[""Theophylline"", ""Azithromycin"", ""Doxofylline""]",Mid,2025-12-01,11:30,2025-12-01T11:30:00,2025-12-01T11:40:00,2025-12-01 11:40,10.0,6.698778,13.397557,
6,209ae35d,M002,Ms. Priya Nair,East Zone,30.0,D013,Dr. A. Bhattacharya,Gynaecology,Dydrogesterone; Progesterone SG; Folic Acid+Iron,"[""Dydrogesterone"", ""Progesterone SG"", ""Folic A...",Low,2025-12-01,13:30,2025-12-01T13:30:00,2025-12-01T13:37:00,2025-12-01 13:37,7.0,4.328169,8.656337,
7,c1b1d6ba,M002,Ms. Priya Nair,East Zone,30.0,D014,Dr. F. Khan,Dermatology,Ketoconazole; Tacrolimus; Terbinafine,"[""Ketoconazole"", ""Tacrolimus"", ""Terbinafine""]",Low,2025-12-01,15:30,2025-12-01T15:30:00,2025-12-01T15:37:00,2025-12-01 15:37,7.0,2.921692,5.843385,
8,0f744a3a,M003,Mr. Rohit Das,South Zone,35.0,D006,Dr. M. Rao,Cardiology,Sacubitril/Valsartan; Rivaroxaban; Furosemide,"[""Sacubitril/Valsartan"", ""Rivaroxaban"", ""Furos...",Mid,2025-12-01,10:00,2025-12-01T10:00:00,2025-12-01T10:10:00,2025-12-01 10:10,10.0,NaN,NaN,
9,64388485,M003,Mr. Rohit Das,South Zone,35.0,D008,Dr. L. Bose,Pediatrics,Montelukast+Levocetirizine Syrup; Cefpodoxime ...,"[""Montelukast+Levocetirizine Syrup"", ""Cefpodox...",Mid,2025-12-01,12:00,2025-12-01T12:00:00,2025-12-01T12:10:00,2025-12-01 12:10,10.0,6.603880,11.320937,


## new Streamlit Dashboard (Final Plan + MR Notes + Visit & RCPA Forms)

This cell writes `streamlit_app.py`, which powers the complete SFA dashboard.  
The Streamlit app performs the following functions:

- Loads and displays the final enriched MR visit plan **from `output/next_7day_mr_visit.csv`**.
- Shows daily/weekly KPIs, filters, searchable tables, and download buttons.
- Provides an **MR Notes Editor** that updates:
  - `mr_notes`
  - `mr_notes_updated_by`
  - `mr_notes_updated_at`  
  These updates are written back **in-place** with an automatic `.bak` backup.
- Displays the travel route map (`output/mr_travel_map.html`) —  
  **Note:** Some browsers or environments may not fully support embedded Folium maps.
- Includes a **Physician Visit Logging Form**, which safely appends new rows to `data/physician_visit_table.csv`.
- Includes an **RCPA (Chemist Audit) Form**, which safely appends new rows to `data/rcpa_chemist_table.csv`.

After saving this cell, launch the dashboard locally:

**Requirements:**  
Install:
- `streamlit`
- `openpyxl`
- `folium` (optional, required for the map)

Make sure the `output/` directory contains the generated final plan and metrics before running the dashboard.


In [ ]:
# FINAL CELL — writes complete streamlit_app.py safely
# (Does NOT run streamlit inside Jupyter)

streamlit_code = r'''
import streamlit as st
import pandas as pd
from pathlib import Path
import json
import tempfile, shutil, os, uuid
from datetime import datetime, date
from typing import Optional

#DIRECTORIES
DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(exist_ok=True)

PHOTO_DIR = DATA_DIR / "photo_proofs"
PHOTO_DIR.mkdir(exist_ok=True)

# FILE PATHS
PLAN_CSV = OUTPUT_DIR / "next_7day_mr_visit.csv"
METRICS_JSON = OUTPUT_DIR / "plan_metrics.json"
MAP_FILE = OUTPUT_DIR / "mr_travel_map.html"

PHYS_VISIT_CSV = DATA_DIR / "physician_visit_table.csv"
RCPA_CSV = DATA_DIR / "rcpa_chemist_table.csv"
MRS_CSV = DATA_DIR / "mr_reps.csv"

BACKUP_SUFFIX = ".bak.csv"

# PAGE CONFIG
st.set_page_config(page_title="Pharma SFA – Final MR Visit Plan", layout="wide")
st.title("Pharma Sales Force Automation — Final MR Visit Plan")

# SAFE HELPERS
def safe_write_df(df: pd.DataFrame, path: Path):
    """Backup + atomic write"""
    try:
        if path.exists():
            backup = path.with_suffix(path.suffix + BACKUP_SUFFIX)
            shutil.copy2(path, backup)
    except:
        pass
    with tempfile.NamedTemporaryFile(mode="w", delete=False, dir=str(path.parent), suffix=".tmp") as tmp:
        tmp_name = tmp.name
        df.to_csv(tmp_name, index=False)
    shutil.move(tmp_name, str(path))

def safe_append_row(path: Path, row: dict):
    if not path.exists():
        pd.DataFrame([row]).to_csv(path, index=False)
        return
    df = pd.read_csv(path)
    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    safe_write_df(df, path)

def save_uploaded_file(uploaded_file, dest_dir: Path) -> Optional[str]:
    if not uploaded_file:
        return ""
    fname = f"{datetime.now().strftime('%Y%m%d_%H%M%S')}_{uuid.uuid4().hex[:6]}_{uploaded_file.name}"
    dest = dest_dir / fname
    with open(dest, "wb") as f:
        f.write(uploaded_file.getbuffer())
    return str(dest)

# LOAD FINAL PLAN
if not PLAN_CSV.exists():
    st.warning("Final plan not found. Run the notebook pipeline to generate output/next_7day_mr_visit.csv.")
    st.stop()

df = pd.read_csv(PLAN_CSV)

# MR name mapping fallback
if "mr_name" not in df.columns or df["mr_name"].isnull().all():
    if MRS_CSV.exists():
        mrs_df = pd.read_csv(MRS_CSV)
        mrs_map = dict(zip(mrs_df["mr_id"], mrs_df["mr_name"]))
        df["mr_name"] = df["assigned_mr"].map(mrs_map)
    else:
        df["mr_name"] = df["assigned_mr"]

# Ensure notes columns exist
for col in ["mr_notes", "mr_notes_updated_by", "mr_notes_updated_at"]:
    if col not in df.columns:
        df[col] = ""

# KPI TILES
col1, col2, col3, col4 = st.columns(4)
with col1: st.metric("Total visits", int(df.shape[0]))
with col2: st.metric("Unique doctors", int(df["doctor_id"].nunique()))
with col3: st.metric("Unique MRs", int(df["assigned_mr"].nunique()))
with col4: st.metric("High priority visits", int(df[df["segment"]=="High"].shape[0]) if "segment" in df.columns else 0)

st.markdown("---")

# FILTER PANEL
left, right = st.columns([2,5])

with left:
    seg = st.selectbox("Segment", ["All"] + sorted(df["segment"].dropna().unique()) if "segment" in df.columns else ["All"])
    therapy = st.selectbox("Therapy Area", ["All"] + sorted(df["therapy_area"].dropna().unique()) if "therapy_area" in df.columns else ["All"])
    mr = st.selectbox("MR", ["All"] + sorted(df["mr_name"].dropna().unique()))
    date_sel = st.selectbox("Date", ["All"] + sorted(df["scheduled_date"].dropna().unique()) if "scheduled_date" in df.columns else ["All"])
    search = st.text_input("Search doctor, molecule or notes")

with right:
    st.write("### Filtered Preview")

df_display = df.copy()
if seg != "All": df_display = df_display[df_display["segment"] == seg]
if therapy != "All": df_display = df_display[df_display["therapy_area"] == therapy]
if mr != "All": df_display = df_display[df_display["mr_name"] == mr]
if date_sel != "All": df_display = df_display[df_display["scheduled_date"] == date_sel]

if search:
    s = search.lower()
    cond = False
    for col in ["doctor_name", "molecules_list", "mr_notes"]:
        if col in df_display.columns:
            cond = cond | df_display[col].astype(str).str.lower().str.contains(s, na=False)
    df_display = df_display[cond]

st.dataframe(df_display.reset_index(drop=True), use_container_width=True)

st.markdown("---")

# TRAVEL MAP
import streamlit.components.v1 as components

st.write("### MR Travel Map")

if MAP_FILE.exists():
    try:
        html = MAP_FILE.read_text(encoding="utf-8")
        components.html(html, height=700, scrolling=True)
    except:
        st.info("Map exists but cannot be embedded. Open output/mr_travel_map.html manually.")
else:
    st.info("Map not found. Regenerate from notebook pipeline.")

st.markdown("---")

# MR SUMMARY
st.write("### MR-level Summary")
try:
    summary = df.groupby(["assigned_mr","mr_name"]).agg(
        num_visits=("doctor_id","count"),
        total_call_minutes=("call_duration_min","sum")
    ).reset_index()
    st.dataframe(summary, use_container_width=True)
except:
    st.info("Summary could not be computed.")

st.markdown("---")

# MR NOTES EDITOR
st.header("Edit MR Notes")

with st.expander("MR Notes Editor", expanded=False):
    df_live = pd.read_csv(PLAN_CSV)
    for col in ["mr_notes","mr_notes_updated_by","mr_notes_updated_at"]:
        if col not in df_live.columns:
            df_live[col] = ""

    pick_mode = st.radio("Select by", ["slot_id","MR name","Doctor name"])

    if pick_mode == "slot_id":
        key_val = st.selectbox("slot_id", [""] + df_live["slot_id"].astype(str).tolist())
        selected = df_live[df_live["slot_id"].astype(str) == key_val]
    elif pick_mode == "MR name":
        key_val = st.selectbox("MR", [""] + sorted(df_live["mr_name"].dropna().unique()))
        selected = df_live[df_live["mr_name"] == key_val]
    else:
        key_val = st.selectbox("Doctor", [""] + sorted(df_live["doctor_name"].dropna().unique()))
        selected = df_live[df_live["doctor_name"] == key_val]

    if selected.empty:
        st.info("Nothing selected.")
    else:
        st.dataframe(selected, use_container_width=True)
        sel_slot = st.selectbox("Select slot", selected["slot_id"].astype(str).tolist())
        idx = df_live.index[df_live["slot_id"].astype(str) == sel_slot].tolist()[0]

        new_note = st.text_area("MR Note", df_live.at[idx,"mr_notes"])
        updated_by = st.text_input("Updated by", "")

        if st.button("Save Note"):
            df_live.at[idx,"mr_notes"] = new_note
            df_live.at[idx,"mr_notes_updated_by"] = updated_by
            df_live.at[idx,"mr_notes_updated_at"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            safe_write_df(df_live, PLAN_CSV)
            st.success("Saved.")
            st.experimental_rerun()

st.markdown("---")

# PHYSICIAN VISIT FORM
st.header("Log Physician Visit")

with st.expander("Physician Visit Form", expanded=False):
    doctors = pd.read_csv(DATA_DIR/"doctors_master.csv") if (DATA_DIR/"doctors_master.csv").exists() else pd.DataFrame()
    mrs = pd.read_csv(MRS_CSV) if MRS_CSV.exists() else pd.DataFrame()

    mr_list = mrs["mr_id"].tolist() if not mrs.empty else []
    doc_list = doctors["doctor_id"].tolist() if not doctors.empty else []

    with st.form("visit_form"):
        mr_id = st.selectbox("MR", mr_list)
        doctor_id = st.selectbox("Doctor", doc_list)
        visit_date = st.date_input("Visit Date", value=date.today())
        visit_time = st.time_input("Time")
        visit_mode = st.selectbox("Mode", ["In-person","Online","Tele-call"])
        product = st.text_input("Products detailed")
        call_duration = st.number_input("Call duration (min)", min_value=1, value=10)
        notes = st.text_area("Core message")
        photo = st.file_uploader("Photo (optional)", type=["jpg","jpeg","png"])
        submit_visit = st.form_submit_button("Save Visit")

        if submit_visit:
            visit_id = f"V{uuid.uuid4().hex[:8]}"
            photo_path = save_uploaded_file(photo, PHOTO_DIR)

            new_row = {
                "visit_id": visit_id,
                "mr_id": mr_id,
                "doctor_id": doctor_id,
                "visit_date": visit_date.strftime("%Y-%m-%d"),
                "visit_time": visit_time.strftime("%H:%M"),
                "visit_mode": visit_mode,
                "product_detailed": product,
                "call_duration_min": call_duration,
                "core_message_shared": notes,
                "photo_proof_url": photo_path
            }

            safe_append_row(PHYS_VISIT_CSV, new_row)
            st.success(f"Saved visit {visit_id}")

st.markdown("---")

# RCPA FORM
st.header("Log RCPA / Chemist Snapshot")

with st.expander("RCPA Form", expanded=False):
    with st.form("rcpa_form"):
        mr_id = st.selectbox("MR", mr_list)
        rcpa_date = st.date_input("RCPA Date", value=date.today())
        chemist_name = st.text_input("Chemist name")
        brand = st.text_input("Brand tracked")
        comp_brand = st.text_input("Competitor brand")
        units = st.number_input("Weekly demand units", min_value=0, value=0)
        remarks = st.text_area("Remarks")
        photo = st.file_uploader("Photo (optional)", type=["jpg","jpeg","png"])
        submit_rcpa = st.form_submit_button("Save RCPA")

        if submit_rcpa:
            rcpa_id = f"R{uuid.uuid4().hex[:8]}"
            photo_path = save_uploaded_file(photo, PHOTO_DIR)
            new_rcpa = {
                "rcpa_id": rcpa_id,
                "mr_id": mr_id,
                "visit_date": rcpa_date.strftime("%Y-%m-%d"),
                "chemist_name": chemist_name,
                "your_brand_name": brand,
                "comp_brand_name": comp_brand,
                "weekly_demand_units": units,
                "remarks": remarks,
                "photo_proof_url": photo_path
            }
            safe_append_row(RCPA_CSV, new_rcpa)
            st.success(f"Saved RCPA {rcpa_id}")

st.markdown("---")
st.write("Backups are auto-created. For multi-user usage, migrate to SQLite/Postgres.")
'''

with open("streamlit_app.py","w",encoding="utf-8") as f:
    f.write(streamlit_code)

print("Wrote streamlit_app.py — Run with:  streamlit run streamlit_app.py")


Wrote streamlit_app.py — Run with:  streamlit run streamlit_app.py
